In [1]:
from platform import python_version
import torch

print('python ver:', python_version(),'torch ver:', torch.__version__)
!which nvcc

python ver: 3.9.13 torch ver: 1.12.1
/c/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.6/bin/nvcc


# PointFlow

In [2]:
from pathlib import Path
cwd = Path.cwd()
module_dir = cwd / 'pytorch_structural_losses'

# pip uninstall PyTorchStructuralLosses
!cd {module_dir} && pip install .

Processing d:\github\pytorch_pcd_metrics\pytorch_structural_losses
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for PyTorchStructuralLosses: filename=PyTorchStructuralLosses-0.1.0-cp39-cp39-win_amd64.whl size=123031 sha256=b09883bedb5bbf0d7b8ae0194c62c6301d2711382f040afafe8f7f5af471b8b1
  Stored in directory: C:\Users\chawa\AppData\Local\Temp\pip-ephem-wheel-cache-_4sjcatk\wheels\52\77\1c\8169621564aa5a5e6e03db17ab422da49e896c7c31049299a8
Successfully built PyTorchStructuralLosses
  Attempting uninstall: PyTorchStructuralLosses
    Found existing installation: PyTorchStructuralLosses 0.1.0
    Uninstalling PyTorchStructuralLosses-0.1.0:
      Successfully uninstalled PyTorchStructuralLosses-0.1.0


In [3]:
import StructuralLosses

a, b = torch.rand((64, 2048, 3)).cuda(), torch.rand((64, 2048, 3)).cuda()
bs = a.shape[0] # batch_size

results = StructuralLosses.compute_all_metrics(a, b, bs)
print('PointFlow metrics w/o JSD')
print(results)

a_, b_ = a.detach().cpu().numpy(), b.detach().cpu().numpy()

jsd_pcd = StructuralLosses.jsd_between_point_cloud_sets(a_, b_)
print('PointFlow JSD:', jsd_pcd)

PointFlow metrics w/o JSD
{'lgan_mmd-CD': tensor(0.0044, device='cuda:0'), 'lgan_cov-CD': tensor(0.4688, device='cuda:0'), 'lgan_mmd_smp-CD': tensor(0.0044, device='cuda:0'), 'lgan_mmd-EMD': tensor(0.0768, device='cuda:0'), 'lgan_cov-EMD': tensor(0.4219, device='cuda:0'), 'lgan_mmd_smp-EMD': tensor(0.0769, device='cuda:0'), '1-NN-CD-acc_t': tensor(0.5469, device='cuda:0'), '1-NN-CD-acc_f': tensor(0.3750, device='cuda:0'), '1-NN-CD-acc': tensor(0.4609, device='cuda:0'), '1-NN-EMD-acc_t': tensor(0.4375, device='cuda:0'), '1-NN-EMD-acc_f': tensor(0.5469, device='cuda:0'), '1-NN-EMD-acc': tensor(0.4922, device='cuda:0')}
PointFlow JSD: 0.0037257533341410465


# MSN-Point-Cloud-Completion


In [4]:
from pathlib import Path
cwd = Path.cwd()
module_dir = cwd / 'pytorch-emd'

# pip uninstall emd
!cd {module_dir} && pip install .

Processing d:\github\pytorch_pcd_metrics\pytorch-emd
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emd: filename=emd-0.0.0-cp39-cp39-win_amd64.whl size=203421 sha256=1ff5123a81ee43e475b51ca18be0861bfc4bb7e3b39de6b034dfc9d06126897d
  Stored in directory: c:\users\chawa\appdata\local\pip\cache\wheels\98\f1\9b\18297e502192368be2d22439c8c1cdb08f6adbfbd64f630887
Successfully built emd
  Attempting uninstall: emd
    Found existing installation: emd 0.0.0
    Uninstalling emd-0.0.0:
      Successfully uninstalled emd-0.0.0


In [5]:
from emd_wrapper import emdModule, test_emd
test_emd()

Input_size:  torch.Size([20, 8192, 3])
Runtime: 2.250506s
EMD: 0.039039
|set(assignment)|: 8192
Verified EMD: 0.039039


In [6]:
a, b = torch.rand((64, 2048, 3)).cuda(), torch.rand((64, 2048, 3)).cuda()
emd_module = emdModule()
eps, iter = 0.002, 10000
dist, assignment = emd_module(a, b, eps, iter)
print(dist.mean(dim=1), assignment[:5])

tensor([0.0050, 0.0046, 0.0048, 0.0046, 0.0050, 0.0054, 0.0050, 0.0048, 0.0048,
        0.0055, 0.0047, 0.0051, 0.0046, 0.0052, 0.0048, 0.0050, 0.0046, 0.0054,
        0.0050, 0.0046, 0.0047, 0.0057, 0.0047, 0.0048, 0.0050, 0.0047, 0.0052,
        0.0057, 0.0046, 0.0051, 0.0051, 0.0047, 0.0048, 0.0047, 0.0055, 0.0048,
        0.0058, 0.0052, 0.0051, 0.0047, 0.0046, 0.0053, 0.0051, 0.0051, 0.0047,
        0.0045, 0.0060, 0.0054, 0.0047, 0.0049, 0.0050, 0.0053, 0.0050, 0.0051,
        0.0058, 0.0050, 0.0047, 0.0052, 0.0048, 0.0052, 0.0051, 0.0047, 0.0048,
        0.0056], device='cuda:0') tensor([[1685,  311, 1441,  ..., 1013, 1002, 1260],
        [1412, 1066, 1000,  ..., 1645,   62,  719],
        [ 883, 1768,  324,  ..., 1005, 1969,  209],
        [1323,  271, 1751,  ...,  363,  255, 1833],
        [1327,  500, 1046,  ..., 1634,  717, 1342]], device='cuda:0',
       dtype=torch.int32)
